# Prompt Optimziation: 프롬프트 최적화

최적화 과정은 프롬프트를 분석하고 재작성하기 위해 협력하는 특화된 AI 에이전트들을 사용하는 다중 에이전트 접근 방식을 사용한다. 이 시스템은 다음과 같은 여러 유형의 일반적인 문제들을 자동으로 식별하고 해결한다:

  - 프롬프트 지시사항 내의 **모순**
  - 누락되거나 불분명한 **형식 명세**
  - 프롬프트와 소수샷(few-shot) 예제 간의 **불일치**

## 1\. 시스템 개요

In [4]:
import os
from dotenv import load_dotenv  

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

MODEL = "gpt-4.1-mini"

In [5]:
# 필요한 모듈들을 임포트한다.
from openai import AsyncOpenAI  # 비동기 OpenAI 클라이언트
import asyncio
import json
import os
from enum import Enum
from typing import Any, List, Dict
from pydantic import BaseModel, Field  # 데이터 유효성 검사를 위한 Pydantic
from agents import Agent, Runner, set_default_openai_client, trace # OpenAI Agents SDK

# 비동기 OpenAI 클라이언트 인스턴스를 저장할 전역 변수다.
openai_client: AsyncOpenAI | None = None

def _get_openai_client() -> AsyncOpenAI:
    """
    OpenAI 클라이언트 인스턴스를 가져오는 함수다.
    이미 생성된 인스턴스가 있으면 그것을 반환하고, 없으면 새로 생성한다. (싱글톤 패턴)
    """
    global openai_client
    if openai_client is None:
        openai_client = AsyncOpenAI()
    return openai_client

# Agents SDK에 기본 OpenAI 클라이언트를 설정한다.
set_default_openai_client(_get_openai_client())

## 2\. 데이터 모델

에이전트 간의 구조화된 통신을 용이하게 하기 위해, 시스템은 Pydantic 모델을 사용하여 입력 및 출력의 예상 형식을 정의한다. 이러한 Pydantic 모델은 데이터를 검증하고 워크플로우 전반에 걸쳐 일관성을 보장하는 데 도움을 준다.

In [6]:
class Role(str, Enum):
    """채팅 메시지의 역할을 정의하는 열거형 클래스다."""
    user = "user"
    assistant = "assistant"

class ChatMessage(BaseModel):
    """소수샷 예제에서 사용되는 단일 채팅 메시지를 나타낸다."""
    role: Role
    content: str

class Issues(BaseModel):
    """검사기(checker)가 반환하는 구조화된 출력이다."""
    has_issues: bool
    issues: List[str]

    @classmethod
    def no_issues(cls) -> "Issues":
        # 문제가 없을 경우 반환할 객체를 생성하는 클래스 메서드다.
        return cls(has_issues=False, issues=[])

class FewShotIssues(Issues):
    """소수샷 모순 감지기의 출력으로, 선택적인 재작성 제안을 포함한다."""
    rewrite_suggestions: List[str] = Field(default_factory=list)

    @classmethod
    def no_issues(cls) -> "FewShotIssues":
        return cls(has_issues=False, issues=[], rewrite_suggestions=[])

class MessagesOutput(BaseModel):
    """`fewshot_rewriter` 에이전트가 반환하는 구조화된 출력이다."""
    messages: list[ChatMessage]


class DevRewriteOutput(BaseModel):
    """재작성기(rewriter)가 정리된 개발자 프롬프트를 반환한다."""
    new_developer_message: str

## 3\. 에이전트 정의

### 에이전트 지시사항의 모범 사례

1.  **명확한 범위 정의**: 각 에이전트는 명시적인 경계를 가진 좁게 정의된 목적을 가진다. 예를 들어, 모순 검사기는 "진정한 자기 모순"에만 초점을 맞추고 "중복이나 군더더기는 모순이 아니다"라고 명시적으로 기술한다.

2.  **단계별 프로세스**: 지시사항은 형식 검사기가 형식 요구사항을 분석하기 전에 먼저 작업을 분류하는 방법과 같이 명확한 방법론을 제공한다.

3.  **명시적 정의**: 핵심 용어는 모호함을 피하기 위해 정확하게 정의된다. 소수샷 일관성 검사기는 준수(compliance)가 정확히 무엇을 의미하는지 설명하는 상세한 "준수 평가 기준(Compliance Rubric)"을 포함한다.

4.  **경계 설정**: 지시사항은 에이전트가 해서는 안 될 일을 명시한다. 소수샷 검사기는 문제를 과도하게 표시하는 것을 방지하기 위해 "범위 외(Out-of-scope)" 항목을 명시적으로 나열한다.

5.  **구조화된 출력 요구사항**: 각 에이전트는 예시와 함께 엄격하게 정의된 출력 형식을 가지며, 최적화 파이프라인의 일관성을 보장한다.

이러한 원칙들은 최적화 시스템에서 효과적으로 함께 작동하는 신뢰성 있고 집중된 에이전트를 만들어낸다. 아래에서 상세한 지시사항이 포함된 전체 에이전트 정의를 볼 수 있다.

In [7]:
dev_contradiction_checker = Agent(
    name="contradiction_detector",
    model=MODEL,
    output_type=Issues,
    instructions="""
    당신은 **Dev-Contradiction-Checker**다.

    목표
    `DEVELOPER_MESSAGE` 변수에 제공된 개발자 프롬프트 **내부**의 *진정한* 자기 모순이나 불가능한 점을 감지한다.

    정의
    • 모순 = 양립할 수 없는 두 개의 절이다.
    • DEVELOPER_MESSAGE 내의 중복이나 군더더기는 모순이 *아니다*.

    필수 작업
    1. 모든 명령/금지 사항을 다른 모든 사항과 비교한다.
    2. 최대 5개의 모순을 나열한다(각각 하나의 글머리 기호로).
    3. 모순이 없으면 그렇다고 말한다.

    출력 형식 (**엄격한 JSON**)
    `Issues` 스키마와 일치하는 객체**만** 반환한다:

    ```json
    {"has_issues": <bool>,
    "issues": [
        "<bullet 1>",
        "<bullet 2>"
    ]
    }
    
    - has_issues = issues 배열이 비어 있지 않을 경우에만 true다.
    - 추가 키, 주석 또는 마크다운을 추가하지 마라.
""",
)
format_checker = Agent(
    name="format_checker",
    model=MODEL,
    output_type=Issues,
    instructions="""
    당신은 Format-Checker다.

    과업
    개발자 프롬프트가 구조화된 출력(JSON/CSV/XML/마크다운 테이블 등)을 요구하는지 결정한다.
    만약 그렇다면, 해당 형식의 누락되거나 불분명한 측면을 지적한다.

    단계
    과업을 다음 중 하나로 분류한다:
    a. "conversation_only" (대화만), 또는
    b. "structured_output_required" (구조화된 출력 필요).

    (b)의 경우:
    - 누락된 필드, 모호한 데이터 타입, 명시되지 않은 순서, 또는 누락된 오류 처리를 지적한다.

    확실하지 않은 경우 문제를 만들어내지 마라. 형식 문제를 지적할 때는 좀 더 보수적으로 접근하라.

    출력 형식
    Issues 스키마를 따르는 엄격하게 유효한 JSON을 반환한다:

    {
    "has_issues": <bool>,
    "issues": ["<desc 1>", "..."]
    }
    최대 5개의 문제를 지적한다. 추가 키나 텍스트는 없다.
""",
)
fewshot_consistency_checker = Agent(
    name="fewshot_consistency_checker",
    model=MODEL,
    output_type=FewShotIssues,
    instructions="""
    당신은 FewShot-Consistency-Checker다.

    목표
    DEVELOPER_MESSAGE 규칙과 함께 제공되는 **어시스턴트** 예제 사이의 충돌을 찾는다.

    USER_EXAMPLES:      <모든 사용자 대화 라인>     # 맥락 정보로만 사용
    ASSISTANT_EXAMPLES: <모든 어시스턴트 대화 라인> # 평가 대상

    방법
    DEVELOPER_MESSAGE에서 핵심 제약 조건을 추출한다:
    - 어조 / 스타일
    - 금지되거나 요구되는 내용
    - 출력 형식 요구사항

    준수 평가 기준 - 주의 깊게 읽어라
    개발자 메시지가 명시적으로 요구하는 것만 평가한다.

    존재할 경우 반드시 확인해야 하는 객관적 제약 조건:
    - 요구되는 출력 타입 구문 (예: "JSON 객체", "단일 문장", "제목 줄").
    - 엄격한 제한 (길이 ≤ N자, 요구되는 언어는 영어, 금지된 단어 등).
    - 개발자가 명시적으로 이름 붙인 필수 토큰이나 필드.

    범위 외 (지적하지 말 것):
    - 응답이 "일반적으로 들리는지", "프롬프트를 반복하는지", 또는 "사용자의 요청을 완전히 반영하는지" - 개발자 텍스트가 명시적으로 그러한 품질을 요구하지 않는 한.
    - 명시되지 않은 창의적인 스타일, 마케팅 품질, 또는 내용의 깊이.
    - 명시적인 규칙을 위반하지 않는 사소한 스타일 선택 (대소문자, 구두점).

    합격/불합격 규칙
    - 어시스턴트 응답이 모든 객관적 제약 조건을 만족하면, 개인적으로는 단조롭거나 관련성이 낮다고 생각하더라도 준수한 것으로 본다.
    - 구체적이고 인용된 규칙이 위반될 때만 문제를 기록한다.

    어시스턴트 목록이 비어 있으면 ⇒ 즉시 has_issues=false를 반환한다.

    각 어시스턴트 예제에 대해:
    - USER_EXAMPLES는 맥락을 위한 것이므로, 준수 여부를 판단하는 데 절대 사용하지 마라.
    - 각 어시스턴트 응답은 오직 개발자 메시지에서 추출한 명시적인 제약 조건에 대해서만 판단한다.
    - 응답이 특정, 인용된 규칙을 위반하면 어떤 규칙을 위반했는지 설명하는 한 줄을 추가한다.
    - 선택적으로, 한 문장으로 된 재작성을 제안할 수 있다 (rewrite_suggestions에 추가).
    - 불확실한 경우, 문제를 지적하지 마라.
    - 보수적으로 판단하라—불확실하거나 모호한 경우는 문제가 아니다.

    소수샷 모순 문제를 지적할 때는 좀 더 보수적으로 접근하라.

    출력 형식
    FewShotIssues와 일치하는 JSON을 반환한다:

    {
    "has_issues": <bool>,
    "issues": ["<explanation 1>", "..."],
    "rewrite_suggestions": ["<suggestion 1>", "..."] // []일 수 있음
    }
    두 배열 모두 최대 5개 항목을 나열한다.
    아무것도 없을 때는 빈 배열을 제공한다.
    마크다운이나 추가 키는 없다.
    """,
)
dev_rewriter = Agent(
    name="dev_rewriter",
    model=MODEL,
    output_type=DevRewriteOutput,
    instructions="""
    당신은 Dev-Rewriter다.

    당신은 다음을 받는다:
    - ORIGINAL_DEVELOPER_MESSAGE
    - CONTRADICTION_ISSUES (비어 있을 수 있음)
    - FORMAT_ISSUES (비어 있을 수 있음)

    재작성 규칙
    원래의 의도와 기능을 보존한다.

    각 모순을 해결한다:
    - 메시지 의도를 보존하는 절을 유지하고, 충돌하는 절은 제거/병합한다.

    FORMAT_ISSUES가 비어 있지 않은 경우:
    - 스키마를 명확하게 정의하거나 명시적인 예제를 제공하는 '## 출력 형식'이라는 새 섹션을 추가한다.

    소수샷 예제는 변경하지 마라.

    새로운 정책이나 범위를 추가하지 마라.

    출력 형식 (엄격한 JSON)
    {
    "new_developer_message": "<전체 재작성된 텍스트>"
    }
    다른 키나 마크다운은 없다.
""",
)
fewshot_rewriter = Agent(
    name="fewshot_rewriter",
    model=MODEL,
    output_type=MessagesOutput,
    instructions="""
    당신은 FewShot-Rewriter다.

    입력 페이로드
    - NEW_DEVELOPER_MESSAGE (이미 최적화됨)
    - ORIGINAL_MESSAGES (사용자/어시스턴트 사전 목록)
    - FEW_SHOT_ISSUES (비어 있지 않음)

    과업
    문제가 지적된 어시스턴트 부분만 다시 생성한다.
    사용자 메시지는 동일하게 유지해야 한다.
    다시 생성된 모든 어시스턴트 응답은 NEW_DEVELOPER_MESSAGE를 반드시 준수해야 한다.

    각 어시스턴트 응답을 다시 생성한 후 확인한다:
    - NEW_DEVELOPER_MESSAGE와 일치하는지 확인하라. 이것이 사실인지 반드시 보장하라.

    출력 형식
    MessagesOutput 스키마와 일치하는 엄격한 JSON을 반환한다:

    {
    "messages": [
        {"role": "user", "content": "..."},
        {"role": "assistant", "content": "..."}
        ]
    }
    지침
    - 원래 순서와 총 개수를 보존한다.
    - 문제가 없었던 메시지는 변경 없이 복사한다.
    """,
)

## 4\. 평가를 통한 에이전트 도출

```json
[
  {
    "focus": "contradiction_issues",
    "input_payload": {
      "developer_message": "항상 **영어**로 답하라.\\n절대 영어로 답하지 마라.",
      "messages": [
        {
          "role": "user",
          "content": "¿Qué hora es?"
        }
      ]
    },
    "golden_output": {
      "changes": true,
      "new_developer_message": "항상 **영어**로 답하라.",
      "new_messages": [
        {
          "role": "user",
          "content": "¿Qué hora es?"
        }
      ],
      "contradiction_issues": "개발자 메시지가 영어로 답하라고 주장하면서 동시에 금지하고 있다.",
      "few_shot_contradiction_issues": "",
      "format_issues": "",
      "general_improvements": ""
    }
  },
  {
    "focus": "few_shot_contradiction_issues",
    "input_payload": {
      "developer_message": "설명 없이 **'yes' 또는 'no'**로만 응답하라.",
      "messages": [
        {
          "role": "user",
          "content": "하늘은 파란가?"
        },
        {
          "role": "assistant",
          "content": "네, 파장의 원리 때문에..."
        },
        {
          "role": "user",
          "content": "물은 젖어 있는가?"
        },
        {
          "role": "assistant",
          "content": "네."
        }
      ]
    },
    "golden_output": {
      "changes": true,
      "new_developer_message": "오직 단일 단어 \\\"yes\\\" 또는 \\\"no\\\"로만 응답하라.",
      "new_messages": [
        {
          "role": "user",
          "content": "하늘은 파란가?"
        },
        {
          "role": "assistant",
          "content": "yes"
        },
        {
          "role": "user",
          "content": "물은 젖어 있는가?"
        },
        {
          "role": "assistant",
          "content": "yes"
        }
      ],
      "contradiction_issues": "",
      "few_shot_contradiction_issues": "지시사항에도 불구하고 어시스턴트 예제에 설명이 포함되어 있다.",
      "format_issues": "",
      "general_improvements": ""
    }
  }
]
```

## 5\. 최적화 워크플로우 실행

최적화 시스템이 실제로 어떻게 종단 간(end-to-end)으로 작동하는지 살펴보자. 핵심 워크플로우는 프롬프트를 효율적으로 처리하고 최적화하기 위해 에이전트들을 병렬로 여러 번 실행하는 것으로 구성된다.

In [10]:
def _normalize_messages(messages: List[Any]) -> List[Dict[str, str]]:
    """Pydantic 메시지 모델 목록을 JSON 직렬화 가능한 사전 목록으로 변환한다."""
    result = []
    for m in messages:
        if hasattr(m, "model_dump"):
            result.append(m.model_dump())
        elif isinstance(m, dict) and "role" in m and "content" in m:
            result.append({"role": str(m["role"]), "content": str(m["content"])})
    return result

async def optimize_prompt_parallel(
    developer_message: str,
    messages: List["ChatMessage"],
) -> Dict[str, Any]:
    """
    모순, 형식, 소수샷 검사기를 병렬로 실행한 다음,
    필요한 경우 프롬프트/예제를 재작성한다.
    API나 엔드포인트에 적합한 통합된 사전을 반환한다.
    """

    with trace("optimize_prompt_workflow"):
        # 1. 모든 검사기를 병렬로 실행한다 (모순, 형식, 예제가 있는 경우 소수샷).
        tasks = [
            Runner.run(dev_contradiction_checker, developer_message),
            Runner.run(format_checker, developer_message),
        ]
        if messages:
            fs_input = {
                "DEVELOPER_MESSAGE": developer_message,
                "USER_EXAMPLES": [m.content for m in messages if m.role == "user"],
                "ASSISTANT_EXAMPLES": [m.content for m in messages if m.role == "assistant"],
            }
            # asyncio.gather를 사용하여 비동기 작업을 병렬로 실행한다.
            tasks.append(Runner.run(fewshot_consistency_checker, json.dumps(fs_input)))

        results = await asyncio.gather(*tasks)

        # 2. 결과의 압축을 푼다.
        cd_issues: Issues = results[0].final_output
        fi_issues: Issues = results[1].final_output
        fs_issues: FewShotIssues = results[2].final_output if messages else FewShotIssues.no_issues()

        # 3. 필요에 따라 재작성한다.
        final_prompt = developer_message
        if cd_issues.has_issues or fi_issues.has_issues:
            # 개발자 프롬프트 재작성 에이전트를 실행한다.
            pr_input = {
                "ORIGINAL_DEVELOPER_MESSAGE": developer_message,
                "CONTRADICTION_ISSUES": cd_issues.model_dump(),
                "FORMAT_ISSUES": fi_issues.model_dump(),
            }
            pr_res = await Runner.run(dev_rewriter, json.dumps(pr_input))
            final_prompt = pr_res.final_output.new_developer_message

        final_messages: list[ChatMessage] | list[dict[str, str]] = messages
        if fs_issues.has_issues:
            # 소수샷 예제 재작성 에이전트를 실행한다.
            mr_input = {
                "NEW_DEVELOPER_MESSAGE": final_prompt,
                "ORIGINAL_MESSAGES": _normalize_messages(messages),
                "FEW_SHOT_ISSUES": fs_issues.model_dump(),
            }
            mr_res = await Runner.run(fewshot_rewriter, json.dumps(mr_input))
            final_messages = mr_res.final_output.messages

        # 4. 최종 결과를 통합된 사전 형태로 반환한다.
        return {
            "changes": True,
            "new_developer_message": final_prompt,
            "new_messages": _normalize_messages(final_messages),
            "contradiction_issues": "\n".join(cd_issues.issues),
            "few_shot_contradiction_issues": "\n".join(fs_issues.issues),
            "format_issues": "\n".join(fi_issues.issues),
        }

## 6\. Example

### 예제 1: 모순 수정하기

In [12]:
async def example_contradiction():
    # 모순된 지시사항을 가진 프롬프트다.
    prompt = """Quick-Start Card — 제품 파서

    목표
    전자상거래 제품 상세 페이지의 원시 HTML을 소화하여 항목을 설명하는 **간결하고 축소된 JSON**을 출력한다.

    **필수 필드:**
    name | brand | sku | price.value | price.currency | images[] | sizes[] | materials[] | care_instructions | features[]

    **추출 우선순위:**
    1. schema.org/JSON-LD 블록
    2. <meta> & 마이크로데이터 태그
    3. 보이는 DOM 대체 (클래스 힌트: "product-name", "price")

    **규칙:**
    - *어떤* 필수 필드라도 누락되면, 즉시 중단하고 `{\"error\": \"FIELD_MISSING:<field>\"}`를 출력한다.
    - 가격: 점(dot)을 사용한 숫자 소수점; 숫자가 아닌 문자 제거 (예: "1.299,00 EUR" → 1299.00 + "EUR").
    - 쿼리 문자열만 다른 이미지 중복 제거. 최고 해상도 10개 이하 유지.
    - 사이즈: 단위 태그("EU", "US") 확인 및 오름차순 정렬.
    - 소재: 첫 글자 대문자화 및 동의어 통합 (예: "polyester 100%" → "Polyester").

    **샘플 구조 (축소됨):**
    ```json
    {\"name\":\"\",\"brand\":\"\",\"sku\":\"\",\"price\":{\"value\":0,\"currency\":\"USD\"},\"images\":[\"\"],\"sizes\":[],\"materials\":[],\"care_instructions\":\"\",\"features\":[]}
    참고: 오류 대신 누락된 필드에 대해 null을 출력하는 것이 허용된다 ###"""

    result = await optimize_prompt_parallel(prompt, [])

    # 결과를 표시한다.
    if result["contradiction_issues"]:
        print("모순점:")
        print(result["contradiction_issues"])
        print()

    print("최적화된 프롬프트:")
    print(result["new_developer_message"])

# 예제를 실행한다.
await example_contradiction()

모순점:
규칙에 '어떤 필수 필드라도 누락되면 즉시 중단하고 {"error": "FIELD_MISSING:<field>"}를 출력한다'고 명시했으나, 하단의 참조 문구에서는 '누락된 필드에 대해 null을 출력하는 것이 허용된다'고 상충되는 지침이 있다.
추출 우선순위가 'schema.org/JSON-LD 블록 → <meta> 및 마이크로데이터 태그 → 보이는 DOM 대체'로 지정되어 있으나, 규칙이나 목표 내에서 이 순서 간의 예외나 조건에 대한 명확한 설명이 없어 모순 가능성 있음.
가격 필드 처리 규칙에서 숫자가 아닌 문자 제거 후 점(dot) 소수점 사용을 요구하면서도, 예시 변환 ('1.299,00 EUR' → 1299.00, 'EUR')에서 소수점과 천 단위 구분 문자의 중복 처리 방법이 애매하게 제시되어 일관성 문제가 발생할 수 있음.
이미지 중복 제거 조건에서 '쿼리 문자열만 다른 경우'만 제거하라고 하면서, 상위 10개 최고 해상도 이미지만 유지하라는 규칙이 혼선 가능성을 야기할 수 있음 (중복 제거 후 개수 카운팅 시점 불명확).
사이즈 정렬 규칙에서 단위 태그('EU', 'US')를 확인한 뒤 '오름차순 정렬'만 요구하는데, 서로 다른 단위가 혼재할 경우 어떤 기준으로 정렬해야 하는지 정의되지 않아 일관성 문제 발생 가능.

최적화된 프롬프트:
Quick-Start Card — 제품 파서

목표
전자상거래 제품 상세 페이지의 원시 HTML을 소화하여 항목을 설명하는 결정하고 추출된 JSON을 출력한다.

필수 필드:
name | brand | sku | price.value | price.currency | images[] | sizes[] | materials[] | care_instructions | features[]

추천 우선순위:
1. schema.org/JSON-LD 블록
2. <meta> 및 마이크로데이터 태그
3. 보이는 DOM 태그 (클래스 힌트: "product-name", "price")

구현 지침:
- 모든 필수 필

> **설명**: 위 코드 실행 결과, 시스템은 "필수 필드가 없으면 에러를 출력하라"는 규칙과 "누락된 필드에 대해 null을 출력해도 된다"는 규칙 사이의 **모순**을 발견한다. `dev_rewriter` 에이전트는 이 모순을 해결하기 위해 후자의 규칙을 제거하고, 출력 형식 섹션을 추가하여 에러 처리 규칙을 명확하게 다시 명시함으로써 프롬프트를 **최적화**했다.

이 예는 시스템이 어떻게 모델에 혼란을 주거나 일관성 없는 출력을 유발할 수 있는 중대한 모순을 감지하고 해결할 수 있는지를 보여준다.

### 예제 2: 프롬프트와 소수샷 예제 간의 불일치 수정하기

In [14]:
async def example_fewshot_fix():
    # 프롬프트: `city`(문자열)와 `population`(정수) 키를 사용하는 JSON으로만 응답하라.
    prompt = "Respond **only** with JSON using keys `city` (string) and `population` (integer)."

    # 예제 메시지: 첫 번째 어시스턴트 응답이 프롬프트의 JSON 형식 규칙을 따르지 않는다.
    messages = [
        {"role": "user", "content": "미국에서 가장 큰 도시는?"},
        {"role": "assistant", "content": "뉴욕시"},
        {"role": "user", "content": "영국에서 가장 큰 도시는?"},
        {"role": "assistant", "content": "{\"city\":\"런던\",\"population\":9541000}"}
    ]


    print("최적화 전 소수샷 예제:")
    print(f"사용자: {messages[0]['content']}")
    print(f"어시스턴트: {messages[1]['content']}")
    print(f"사용자: {messages[2]['content']}")
    print(f"어시스턴트: {messages[3]['content']}")
    print()

    # 최적화 API를 호출한다.
    result = await optimize_prompt_parallel(prompt, [ChatMessage(**m) for m in messages])

    # 결과를 표시한다.
    if result["few_shot_contradiction_issues"]:
        print("불일치 발견:", result["few_shot_contradiction_issues"])
        print()

    # 최적화된 소수샷 예제를 보여준다.
    optimized_messages = result["new_messages"]
    print("최적화 후 소수샷 예제:")
    print(f"사용자: {optimized_messages[0]['content']}")
    print(f"어시스턴트: {optimized_messages[1]['content']}")
    print(f"사용자: {optimized_messages[2]['content']}")
    print(f"어시스턴트: {optimized_messages[3]['content']}")

# 예제를 실행한다.
await example_fewshot_fix()

최적화 전 소수샷 예제:
사용자: 미국에서 가장 큰 도시는?
어시스턴트: 뉴욕시
사용자: 영국에서 가장 큰 도시는?
어시스턴트: {"city":"런던","population":9541000}

불일치 발견: The first assistant response is '뉴욕시', which is a string alone, not a JSON object with keys `city` and `population` as required.
The second assistant response is a JSON string instead of a JSON object; it's a string containing JSON, which violates the requirement to respond only with a JSON object.

최적화 후 소수샷 예제:
사용자: bf8ad8ed5bd1dcedd7cd3f795bd440ac, bf8adecd3f4f5, e6893f185
어시스턴트: {"city":"d4f36d1f5b2d8c7","population":8419000}
사용자: bd7ecf6,e6617, bd6edcf4, 80e0cdeabedc, 88b5c6, bcdaf2, f3c7, b5f1ccc1, f4b2fb90, 41eeb5f6
어시스턴트: {"city":"d4f36d1f5b2d8c7","population":9541000}


### 예제 3: 긴 프롬프트에서 형식 명확히 하기

In [15]:
async def example_format_issue():
    # 불분명하거나 일관성 없는 형식 지시를 가진 프롬프트다.
    prompt = """과업 → 빽빽한 특허 청구항을 200단어의 일반인용 요약과 용어집으로 번역하라.

    작업 단계:
    1. 세미콜론, "wherein", 또는 번호가 매겨진 하위 조항에서 청구항을 분할한다.
    2. 각 덩어리에 대해:
       a) 목적을 식별한다.
       b) 기술적인 명사를 일상적인 비유로 대체한다.
       c) 양적 제한은 그대로 유지한다 (예: "≥150 C").
    3. 흔하지 않은 과학 용어에 별표를 표시하고, 나중에 정의한다.
    4. 흐름이 있는 문단으로 재구성한다; 청구항의 범위를 넓히거나 좁히지 **마라**.
    5. 법적 의미를 변경하지 않는 상용구는 생략한다.

    출력은 마크다운 템플릿을 따라야 한다:
    - 요약 섹션.
    - 표시된 용어와 그 정의가 있는 용어집 섹션.

    예외 사례:
    - 청구항이 5kB를 초과하면 CLAIM_TOO_LARGE로 응답한다.
    - 청구항 텍스트가 이미 평이한 영어인 경우, 용어집을 생략하고 복잡한 용어가 없다고 명시한다.

    기억하라: 당신은 법적 조언을 제공하는 것이 아니다—이것은 오직 내부 이해를 위한 것이다."""

    # 형식 문제를 확인하기 위해 최적화 API를 호출한다.
    result = await optimize_prompt_parallel(prompt, [])

    # 결과를 표시한다.
    if result.get("format_issues"):
        print("형식 문제 발견:", result["format_issues"])
        print()

    print("최적화된 프롬프트:")
    print(result["new_developer_message"])

# 예제를 실행한다.
await example_format_issue()

형식 문제 발견: 과업 지시가 명확한 구조화된 출력 형식을 명시하지 않음. '마크다운 템플릿'을 언급하지만 구체적인 마크다운 구조(헤더 수준, 섹션 제목 등)와 필드 구성이 불분명함.
출력 예외 처리 조건은 있으나, 예외 상황에서의 구체적인 출력 포맷(예: CLAIM_TOO_LARGE 응답 구조)이 정의되지 않음.
용어집 섹션에 포함할 '표시된 용어와 그 정의'의 데이터 타입이나 형식(예: 목록, 표 등)이 명확하지 않음.
청구항 분할 및 재구성 단계에서 각 단계의 출력 형태(예: 목적 식별 결과, 비유 표현 등)를 어떻게 포함할지에 대한 지침이 부재.
섹션 간의 순서(요약 섹션이 먼저인지, 용어집이 후인지)는 명시되었으나, 섹션 내 구체적 내용 구성에 대한 상세 명세가 누락됨.

최적화된 프롬프트:
과업 → 락된 특허 청구항을 200단어의 일방인용 요약과 용어집으로 번역하라.

작업 단계:
1. 세미콜론, "wherein", 또는는 번호가 매겨진 하위 조항에서 청구항을 분할한다.
2. 각 분할에 대해:
   a) 목적으로 식별한다.
   b) 기술적인 명사를 일상적인 비유로 대체한다.
   c) 양적인 제언은 그대로 유지한다 (예: "≤150 C").
3. 의미가 명확하지 않은 과학 용어에 별표 표시를 하고, 나중에 설명한다.
4. 간결한 문단으로 재구성한다; 청구항 범위를 벗어나거나 너무 좁게 해석하지 말라.
5. 법적 의미를 변형하지 않는 상황에서 명확히 설명한다.

출력은 마크다운 템플릿 형식으로 명확히 구성되어야 한다:
- 요약 섹션.
- 표시된 용어와 그 정확한 뜻을 담은 용어집 섹션.

예외 사항:
- 청구항이 5kB 초과 시 CLAIM_TOO_LARGE 오류가 발생한다.
- 청구항 텍스트가 이미지 포함 또는 부적절한 용어집을 생성할 경우 오류가 발생한다.

기억하라: 당신은 법적 조언을 제공하지 않으며, 이는 오직 내부 이해를 위한 것이다.
